# Journey to Springfield

## Imports

In [22]:
import torch
import numpy as np
from os.path import exists
import PIL
import torchvision.models as models
from torch.utils.data import DataLoader, ConcatDataset
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    !nvidia-smi

CUDA is available!  Training on GPU ...
Mon Sep 18 09:23:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060      WDDM  | 00000000:07:00.0  On |                  N/A |
|  0%   44C    P8              18W / 170W |   2071MiB /  6144MiB |     13%      Default |
|                                         |                      |                  N/A |
+---------------------------

In [3]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


d:\Anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
# разные режимы датасета
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [5]:
TRAIN_DIR = Path('data/train/simpsons_dataset')
TEST_DIR = Path('data/testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

In [6]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

## Functions for training and evaluating

In [7]:
from torchvision.models.inception import InceptionOutputs

def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    model.train()

    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        #<1>
        if isinstance(outputs, InceptionOutputs):
            outputs = torch.nn.functional.softmax(outputs[0], dim=0)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)

    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [8]:
from sklearn.metrics import f1_score
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        f1 = f1_score(labels.data.to('cpu'), preds.to('cpu'), average='micro')
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc, f1

In [9]:
import datetime


def train(train_dl, val_dl, model, epochs, optimizer, batch_size=64):
    best_f1 = 0
    #save_path = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))

    train_loader = train_dl #DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = val_dl #DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "Epoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f}\ntrain_acc {t_acc:0.4f} val_acc {v_acc:0.4f}\n{time}\nF1score {f1}\n"

    print('Training started at', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = optimizer
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)

            val_loss, val_acc, f1 = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))

            if f1 > best_f1:
                save_path = 'model_f1_{}_{}.pth'.format(f1, datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))
                torch.save(model.state_dict(), save_path)
                best_f1 = f1
            
            

            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc,
                                           time=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                           f1=f1))
            
        print('Training end at', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    return history

In [10]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []

        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs.cpu()).cpu()
            logits.append(outputs)

    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [11]:
def show_loss(history):
    loss, acc, val_loss, val_acc = zip(*history)
    plt.figure(figsize=(15, 9))
    plt.plot(loss, label="train_loss")
    plt.plot(val_loss, label="val_loss")
    plt.legend(loc='best')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.show()

In [12]:
n_classes = len(np.unique(train_val_labels))

## SimpsonsDS

In [13]:
class SimpsonsDs(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode, transform=None, image_size=(224, 224)):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        self.image_size = image_size

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)

        self.label_encoder = LabelEncoder()

        if (transform is None): # or (self.mode =='test') or (self.mode =='val') :
            transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(self.image_size, antialias=True)
                        ])
        
        self.transform = transform

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        
        x = self.load_sample(self.files[index])
        #x = self._prepare_sample(x)
        #x = np.array(x / 255, dtype='float32')
        x = self.transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [23]:
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)

        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                                 )
        ])
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

### Functions for calc accuracy per character

In [14]:
uniqe_labels = list(set(train_val_labels))
simpsons_tp_all = {}
for i in uniqe_labels:
    simpsons_tp_all[i] = [0,0]
print(simpsons_tp_all)


{'groundskeeper_willie': [0, 0], 'edna_krabappel': [0, 0], 'agnes_skinner': [0, 0], 'homer_simpson': [0, 0], 'maggie_simpson': [0, 0], 'selma_bouvier': [0, 0], 'otto_mann': [0, 0], 'krusty_the_clown': [0, 0], 'apu_nahasapeemapetilon': [0, 0], 'professor_john_frink': [0, 0], 'troy_mcclure': [0, 0], 'miss_hoover': [0, 0], 'barney_gumble': [0, 0], 'lisa_simpson': [0, 0], 'fat_tony': [0, 0], 'abraham_grampa_simpson': [0, 0], 'marge_simpson': [0, 0], 'comic_book_guy': [0, 0], 'snake_jailbird': [0, 0], 'ned_flanders': [0, 0], 'patty_bouvier': [0, 0], 'sideshow_mel': [0, 0], 'principal_skinner': [0, 0], 'cletus_spuckler': [0, 0], 'waylon_smithers': [0, 0], 'milhouse_van_houten': [0, 0], 'carl_carlson': [0, 0], 'ralph_wiggum': [0, 0], 'martin_prince': [0, 0], 'lenny_leonard': [0, 0], 'sideshow_bob': [0, 0], 'charles_montgomery_burns': [0, 0], 'disco_stu': [0, 0], 'moe_szyslak': [0, 0], 'chief_wiggum': [0, 0], 'nelson_muntz': [0, 0], 'mayor_quimby': [0, 0], 'bart_simpson': [0, 0], 'kent_brockma

In [15]:

def prescision_per_class(model, dataset, batch_size=128):
    simpsons_tp_all = {}
    for i in uniqe_labels:
        simpsons_tp_all[i] = [0,0]

    simpl_dl = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    with torch.no_grad():
        for x, y in simpl_dl:
            
            pred = torch.argmax(model(x), 1)
            for i_pr, i_true in zip(pred, y):
                current_label = dataset.label_encoder.inverse_transform([i_true])[0]
                
                simpsons_tp_all[current_label][0] += (i_pr==i_true)
                simpsons_tp_all[current_label][1] += 1
    return simpsons_tp_all
                
            

In [16]:
def acc_per_character(simpsons_acc_val):
    total = 0
    pre_table = []
    for name, presc in simpsons_acc_val.items():
        acc = 0
        try:
            acc = presc[0]/presc[1]
        except:
            pass
        pre_table.append([name, acc, presc[0], presc[1]])
        '''name += ' '*10
        presc[0] = int(presc[0])
        print(f'{name[0:10]} Acc: {presc[0]/presc[1]:0.4f} {presc[1]}')# {presc[0]:0.4f} / {presc[1]:0.4f}')
        total += presc[0]/presc[1]'''
    #print(total/len(simpsons_acc_val))
    df = pd.DataFrame(pre_table, columns=['Name', 'Acc', 'TP', 'Amount'])
    print(df.sort_values(by='Acc', ascending=False))

# Пора заняться дисбалансом  
код по устранению диспаланса через oversampling был взят у https://stepik.org/users/247846  

Идея простая и элегантная, вместо того, чтобы заниматься устранением дисбаланса через WeightedRandomSampler  
Мы просто дублируе путь к картинкам персонажей, у которых количество экземпляров изначально меньше 100.(И мы просто несколько раз обращаемся к одним и тем же картинка при обучении)

In [17]:
train_labels = [path.parent.name for path in train_files] # классы train
val_labels = [path.parent.name for path in val_files]     # классы val

def create_dct_path_labels(train_files, train_labels):
    dct_simpsons = {}
    for label_i in np.unique(train_labels).tolist():
        dct_simpsons[label_i] = []

    for path_i, label_i in zip(train_files, train_labels):
        dct_simpsons[label_i].append(path_i)

    return dct_simpsons

def print_dct(dct_simpsons):
    for key in dct_simpsons:
        print(f"{key}\t{dct_simpsons[key]}")

In [18]:
dct_path_train = create_dct_path_labels(train_files, train_labels)


In [19]:
increase_to =  300#<4>
for person in dct_path_train:
    if len(dct_path_train[person]) < 100:
        dct_path_train[person] = dct_path_train[person] * (increase_to // len(dct_path_train[person]))
        dct_path_train[person].extend(dct_path_train[person][:increase_to - len(dct_path_train[person])])

In [20]:
# Проверим что получилось 
for person in dct_path_train:
    print(f"{person}\t{len(dct_path_train[person])}")

abraham_grampa_simpson	685
agnes_skinner	300
apu_nahasapeemapetilon	467
barney_gumble	300
bart_simpson	1006
carl_carlson	300
charles_montgomery_burns	895
chief_wiggum	739
cletus_spuckler	300
comic_book_guy	352
disco_stu	300
edna_krabappel	343
fat_tony	300
gil	300
groundskeeper_willie	300
homer_simpson	1684
kent_brockman	373
krusty_the_clown	904
lenny_leonard	233
lisa_simpson	1015
maggie_simpson	300
marge_simpson	968
martin_prince	300
mayor_quimby	185
milhouse_van_houten	809
miss_hoover	300
moe_szyslak	1089
ned_flanders	1090
nelson_muntz	269
otto_mann	300
patty_bouvier	300
principal_skinner	895
professor_john_frink	300
rainier_wolfcastle	300
ralph_wiggum	300
selma_bouvier	300
sideshow_bob	658
sideshow_mel	300
snake_jailbird	300
troy_mcclure	300
waylon_smithers	136


In [21]:
new_train_files = []

for person in dct_path_train:
    new_train_files.extend(dct_path_train[person])

new_train_label = [path.parent.name for path in new_train_files] 

Конец скопированного кода

# Запуск модели

## Main model code

In [ ]:
from torch.utils.data import DataLoader, ConcatDataset
image_size = (224, 224) #<2>
scale_grade = 1.5
image_size_mod = (int(image_size[0]*scale_grade),
                  int(image_size[1]*scale_grade))


transform_basic = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size, antialias=True)
                        ])

transform_main = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size, antialias=True),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

transform_pyramid_center = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size_mod, antialias=True),
                            transforms.CenterCrop(image_size),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

transform_pyramid_rand = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size_mod, antialias=True),
                            transforms.RandomCrop(image_size),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

val_ds = SimpsonsDs(val_files, mode='val')#, transform=transform)

 
train_ds_basic = SimpsonsDs(new_train_files, mode='train', transform=transform_basic)
train_ds_main = SimpsonsDs(new_train_files, mode='train', transform=transform_main)
train_ds_center = SimpsonsDs(new_train_files, mode='train', transform=transform_pyramid_center)
train_ds_rand = SimpsonsDs(new_train_files, mode='train', transform=transform_pyramid_rand)

full_train_dataset = ConcatDataset([train_ds_basic, train_ds_main, train_ds_center, train_ds_rand])

batch_size = 128

big_train_loader = DataLoader(full_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
resnet34_horiz = models.resnet34(pretrained=True)

resnet34_horiz.fc = nn.Linear(512, n_classes)
# Выключаем подсчет градиентов для слоев, которые не будем обучать
for param in resnet34_horiz.parameters():
    param.requires_grad = False

#<4>

for param in resnet34_horiz.layer3.parameters():
    param.requires_grad = True

for param in resnet34_horiz.layer4.parameters():
    param.requires_grad = True

for param in resnet34_horiz.fc.parameters():
    param.requires_grad = True
resnet34_horiz.to(DEVICE)

optimizer = torch.optim.AdamW(resnet34_horiz.parameters(), lr=3e-4)

history = train(big_train_loader, val_loader, model=resnet34_horiz, optimizer=optimizer, epochs=7)#, batch_size=64)

## Эксперемент с разморозокй всех слоев с 1-4 + fc

In [35]:
from torch.utils.data import DataLoader, ConcatDataset
image_size = (224, 224) #<2>
scale_grade = 1.5
image_size_mod = (int(image_size[0]*scale_grade),
                  int(image_size[1]*scale_grade))


transform_basic = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size, antialias=True)
                        ])

transform_main = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size, antialias=True),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

transform_pyramid_center = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size_mod, antialias=True),
                            transforms.CenterCrop(image_size),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

transform_pyramid_rand = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],), 
                            transforms.Resize(image_size_mod, antialias=True),
                            transforms.RandomCrop(image_size),
                            transforms.RandomHorizontalFlip(p=0.5),
                            #transforms.RandomVerticalFlip(p=0.3),
                            transforms.RandomRotation(degrees=(-45, 45))
                        ])

val_ds = SimpsonsDs(val_files, mode='val')#, transform=transform)

 
train_ds_basic = SimpsonsDs(new_train_files, mode='train', transform=transform_basic)
train_ds_main = SimpsonsDs(new_train_files, mode='train', transform=transform_main)
train_ds_center = SimpsonsDs(new_train_files, mode='train', transform=transform_pyramid_center)
train_ds_rand = SimpsonsDs(new_train_files, mode='train', transform=transform_pyramid_rand)

full_train_dataset = ConcatDataset([train_ds_basic, train_ds_main, train_ds_center, train_ds_rand])

batch_size = 128

big_train_loader = DataLoader(full_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
resnet34_horiz = models.resnet34(pretrained=True)

resnet34_horiz.fc = nn.Linear(512, n_classes)
# Выключаем подсчет градиентов для слоев, которые не будем обучать
for param in resnet34_horiz.parameters():
    param.requires_grad = False

#<4>
for param in resnet34_horiz.layer1.parameters():
    param.requires_grad = True

for param in resnet34_horiz.layer2.parameters():
    param.requires_grad = True

for param in resnet34_horiz.layer3.parameters():
    param.requires_grad = True

for param in resnet34_horiz.layer4.parameters():
    param.requires_grad = True

for param in resnet34_horiz.fc.parameters():
    param.requires_grad = True
resnet34_horiz.to(DEVICE)

optimizer = torch.optim.AdamW(resnet34_horiz.parameters(), lr=3e-4)

history = train(big_train_loader, val_loader, model=resnet34_horiz, optimizer=optimizer, epochs=7)#, batch_size=64)

d:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training started at 2023-09-18 09:50:03


epoch:   0%|          | 0/7 [00:00<?, ?it/s]

loss 0.32355257072222865


epoch:  14%|█▍        | 1/7 [23:40<2:22:01, 1420.30s/it]

...

Epoch 005 train_loss: 0.0595     val_loss 0.1787
train_acc 0.9835 val_acc 0.9618
2023-09-18 11:38:50
F1score 0.9469026548672567

loss 0.05782810868115401


epoch:  86%|████████▌ | 6/7 [2:09:22<21:14, 1274.57s/it]

Epoch 006 train_loss: 0.0578     val_loss 0.1772
train_acc 0.9838 val_acc 0.9597
2023-09-18 11:59:26
F1score 0.9292035398230089

loss 0.04947265785896891


epoch: 100%|██████████| 7/7 [2:29:59<00:00, 1285.70s/it]

Epoch 007 train_loss: 0.0495     val_loss 0.1649
train_acc 0.9859 val_acc 0.9664
2023-09-18 12:20:03
F1score 0.9203539823008849

Training end at 2023-09-18 12:20:03


# Loading model and calc final prediction

## Loading model

In [26]:
path_to_model = 'model_f1_0.9557522123893806_2023_09_17_17_26.pth'
resnet34_horiz = models.resnet34()
resnet34_horiz.fc = nn.Linear(512, 42) # тут должно стоять n_classes но я накосячил и стоит 42
resnet34_horiz.load_state_dict(torch.load(path_to_model))
resnet34_horiz.eval() 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Точность каждого класса на val выборке

In [30]:
import pandas as pd

In [28]:
# uncomment if error in the cell below
#val_ds = SimpsonsDs(val_files, mode='val')#, transform=transform)

In [31]:
simpsons_acc_val = prescision_per_class(resnet34_horiz.to('cpu'), val_ds)
acc_per_character(simpsons_acc_val)

                        Name             Acc           TP  Amount
40        rainier_wolfcastle      tensor(1.)   tensor(11)      11
11               miss_hoover      tensor(1.)    tensor(4)       4
26              carl_carlson      tensor(1.)   tensor(24)      24
25       milhouse_van_houten      tensor(1.)  tensor(270)     270
24           waylon_smithers      tensor(1.)   tensor(45)      45
23           cletus_spuckler      tensor(1.)   tensor(12)      12
21              sideshow_mel      tensor(1.)   tensor(10)      10
18            snake_jailbird      tensor(1.)   tensor(14)      14
14                  fat_tony      tensor(1.)    tensor(7)       7
12             barney_gumble      tensor(1.)   tensor(26)      26
34              chief_wiggum      tensor(1.)  tensor(247)     247
10              troy_mcclure      tensor(1.)    tensor(2)       2
5              selma_bouvier      tensor(1.)   tensor(26)      26
8     apu_nahasapeemapetilon      tensor(1.)  tensor(156)     156
2         

## Predict and save csv

In [24]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [32]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=128)
probs = predict(resnet34_horiz, test_loader)


preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]


In [33]:
my_submitt = pd.DataFrame({'Id':test_filenames, 'Expected':preds})
print(my_submitt.head(10))

           Id                Expected
0    img0.jpg            nelson_muntz
1    img1.jpg            bart_simpson
2   img10.jpg            ned_flanders
3  img100.jpg            chief_wiggum
4  img101.jpg  apu_nahasapeemapetilon
5  img102.jpg           kent_brockman
6  img103.jpg          edna_krabappel
7  img104.jpg            chief_wiggum
8  img105.jpg            lisa_simpson
9  img106.jpg           kent_brockman


In [34]:
my_submitt.to_csv('mysub_baseline.csv', index=False)